## Beyond `__syncthreads()`: The Cooperative Groups API

Cooperative Groups provides a flexible, composable API for thread synchronization and collective operations.

In [ ]:
%%writefile cg_basics_advanced.cu
#include <stdio.h>
#include <cooperative_groups.h>
#include <cooperative_groups/reduce.h>

namespace cg = cooperative_groups;

__global__ void cgDemo() {
    // Get thread block group
    cg::thread_block block = cg::this_thread_block();
    
    // Partition into tiles of 32 threads (warp-aligned)
    cg::thread_block_tile<32> warp = cg::tiled_partition<32>(block);
    
    // Partition into tiles of 16 threads
    cg::thread_block_tile<16> half_warp = cg::tiled_partition<16>(block);
    
    // Get coalesced threads (active threads in warp)
    cg::coalesced_group active = cg::coalesced_threads();
    
    if (block.thread_rank() == 0) {
        printf("Block size: %d\n", block.size());
        printf("Warp tile size: %d\n", warp.size());
        printf("Half-warp tile size: %d\n", half_warp.size());
    }
}

int main() {
    cgDemo<<<1, 128>>>();
    cudaDeviceSynchronize();
    return 0;
}

In [ ]:
!nvcc cg_basics_advanced.cu -o cg_basics_advanced && ./cg_basics_advanced

## CG Collective Operations: `reduce()`

Hardware-accelerated reductions with `cg::reduce()` - much faster than manual shuffle reductions!

In [ ]:
%%writefile cg_reduce.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <cooperative_groups.h>
#include <cooperative_groups/reduce.h>

namespace cg = cooperative_groups;

__global__ void cgReduceKernel(int* input, int* output, int n) {
    cg::thread_block block = cg::this_thread_block();
    cg::thread_block_tile<32> warp = cg::tiled_partition<32>(block);
    
    int tid = block.thread_rank();
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load value
    int val = (idx < n) ? input[idx] : 0;
    
    // Warp-level reduction using CG (hardware-accelerated on CC 8.0+)
    int warp_sum = cg::reduce(warp, val, cg::plus<int>());
    
    // First thread in each warp has the sum
    __shared__ int warp_sums[32];  // Max 32 warps per block
    if (warp.thread_rank() == 0) {
        warp_sums[tid / 32] = warp_sum;
    }
    block.sync();
    
    // Final reduction of warp sums (first warp only)
    if (tid < 32) {
        int num_warps = (blockDim.x + 31) / 32;
        val = (tid < num_warps) ? warp_sums[tid] : 0;
        int block_sum = cg::reduce(warp, val, cg::plus<int>());
        
        if (tid == 0) {
            output[blockIdx.x] = block_sum;
        }
    }
}

int main() {
    const int N = 1024;
    int h_input[N], h_output[8];
    
    for (int i = 0; i < N; i++) h_input[i] = 1;  // Sum should be N
    
    int *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(int));
    cudaMalloc(&d_output, 8 * sizeof(int));
    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice);
    
    cgReduceKernel<<<8, 128>>>(d_input, d_output, N);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_output, d_output, 8 * sizeof(int), cudaMemcpyDeviceToHost);
    
    int total = 0;
    for (int i = 0; i < 8; i++) total += h_output[i];
    printf("Sum of %d ones = %d (expected %d)\n", N, total, N);
    
    cudaFree(d_input);
    cudaFree(d_output);
    return 0;
}

In [ ]:
!nvcc cg_reduce.cu -o cg_reduce && ./cg_reduce

## CG Scan Operations: `inclusive_scan()` and `exclusive_scan()`

In [ ]:
%%writefile cg_scan.cu
#include <stdio.h>
#include <cooperative_groups.h>
#include <cooperative_groups/scan.h>

namespace cg = cooperative_groups;

__global__ void cgScanKernel(int* input, int* incl_output, int* excl_output, int n) {
    cg::thread_block block = cg::this_thread_block();
    
    int tid = block.thread_rank();
    int val = (tid < n) ? input[tid] : 0;
    
    // Inclusive scan: each element includes itself
    // Input:  [1, 2, 3, 4]
    // Output: [1, 3, 6, 10]
    int incl = cg::inclusive_scan(block, val, cg::plus<int>());
    
    // Exclusive scan: each element excludes itself
    // Input:  [1, 2, 3, 4]
    // Output: [0, 1, 3, 6]
    int excl = cg::exclusive_scan(block, val, cg::plus<int>());
    
    if (tid < n) {
        incl_output[tid] = incl;
        excl_output[tid] = excl;
    }
}

int main() {
    const int N = 8;
    int h_input[N] = {1, 2, 3, 4, 5, 6, 7, 8};
    int h_incl[N], h_excl[N];
    
    int *d_input, *d_incl, *d_excl;
    cudaMalloc(&d_input, N * sizeof(int));
    cudaMalloc(&d_incl, N * sizeof(int));
    cudaMalloc(&d_excl, N * sizeof(int));
    
    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice);
    
    cgScanKernel<<<1, N>>>(d_input, d_incl, d_excl, N);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_incl, d_incl, N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_excl, d_excl, N * sizeof(int), cudaMemcpyDeviceToHost);
    
    printf("Input:          ");
    for (int i = 0; i < N; i++) printf("%3d ", h_input[i]);
    printf("\nInclusive scan: ");
    for (int i = 0; i < N; i++) printf("%3d ", h_incl[i]);
    printf("\nExclusive scan: ");
    for (int i = 0; i < N; i++) printf("%3d ", h_excl[i]);
    printf("\n");
    
    cudaFree(d_input);
    cudaFree(d_incl);
    cudaFree(d_excl);
    return 0;
}

In [ ]:
!nvcc cg_scan.cu -o cg_scan && ./cg_scan

## Warp-Aggregated Atomics Pattern

Reduce atomic contention by having one thread per warp perform the atomic operation.

In [ ]:
%%writefile cg_warp_atomics.cu
#include <stdio.h>
#include <cooperative_groups.h>
#include <cooperative_groups/reduce.h>

namespace cg = cooperative_groups;

// Naive: every thread does atomic - HIGH CONTENTION
__device__ int naiveAtomicInc(int* counter) {
    return atomicAdd(counter, 1);
}

// Optimized: warp-aggregated atomic - MUCH FASTER
__device__ int warpAggregatedAtomicInc(int* counter) {
    cg::coalesced_group active = cg::coalesced_threads();
    
    int warp_res;
    if (active.thread_rank() == 0) {
        // Only leader does the atomic
        warp_res = atomicAdd(counter, active.size());
    }
    
    // Broadcast result and compute individual values
    warp_res = active.shfl(warp_res, 0);
    return warp_res + active.thread_rank();
}

__global__ void testNaive(int* counter, int* results, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        results[idx] = naiveAtomicInc(counter);
    }
}

__global__ void testWarpAgg(int* counter, int* results, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        results[idx] = warpAggregatedAtomicInc(counter);
    }
}

int main() {
    const int N = 1000000;
    int *d_counter, *d_results;
    
    cudaMalloc(&d_counter, sizeof(int));
    cudaMalloc(&d_results, N * sizeof(int));
    
    // Benchmark naive
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaMemset(d_counter, 0, sizeof(int));
    cudaEventRecord(start);
    testNaive<<<(N+255)/256, 256>>>(d_counter, d_results, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float naive_ms;
    cudaEventElapsedTime(&naive_ms, start, stop);
    
    // Benchmark warp-aggregated
    cudaMemset(d_counter, 0, sizeof(int));
    cudaEventRecord(start);
    testWarpAgg<<<(N+255)/256, 256>>>(d_counter, d_results, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float warp_ms;
    cudaEventElapsedTime(&warp_ms, start, stop);
    
    printf("Naive atomics:          %.3f ms\n", naive_ms);
    printf("Warp-aggregated atomics: %.3f ms\n", warp_ms);
    printf("Speedup: %.1fx\n", naive_ms / warp_ms);
    
    cudaFree(d_counter);
    cudaFree(d_results);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}

In [ ]:
!nvcc cg_warp_atomics.cu -o cg_warp_atomics && ./cg_warp_atomics

## Grid-Level Synchronization

Cooperative kernel launch allows synchronizing ALL threads across the entire grid.

In [ ]:
%%writefile cg_grid_sync.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <cooperative_groups.h>

namespace cg = cooperative_groups;

__global__ void gridSyncKernel(int* data, int n) {
    cg::grid_group grid = cg::this_grid();
    
    int idx = grid.thread_rank();
    
    // Phase 1: Each thread writes its index
    if (idx < n) {
        data[idx] = idx;
    }
    
    // Grid-wide synchronization - ALL threads wait here
    grid.sync();
    
    // Phase 2: Read neighbor's value (guaranteed to be written)
    if (idx < n && idx > 0) {
        data[idx] += data[idx - 1];
    }
    
    if (idx == 0) {
        printf("Grid sync completed across %lu threads\n", grid.size());
    }
}

int main() {
    // Check for cooperative launch support
    int dev = 0;
    int supportsCoopLaunch = 0;
    cudaDeviceGetAttribute(&supportsCoopLaunch, cudaDevAttrCooperativeLaunch, dev);
    
    if (!supportsCoopLaunch) {
        printf("Device does not support cooperative launch\n");
        return 0;
    }
    
    const int N = 1024;
    int* d_data;
    cudaMalloc(&d_data, N * sizeof(int));
    
    // Use cooperative launch
    void* args[] = {&d_data, (void*)&N};
    
    int numBlocks;
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, gridSyncKernel, 128, 0);
    
    int numSMs;
    cudaDeviceGetAttribute(&numSMs, cudaDevAttrMultiProcessorCount, dev);
    
    dim3 grid(numSMs * numBlocks);
    dim3 block(128);
    
    cudaLaunchCooperativeKernel((void*)gridSyncKernel, grid, block, args);
    cudaDeviceSynchronize();
    
    int h_data[N];
    cudaMemcpy(h_data, d_data, N * sizeof(int), cudaMemcpyDeviceToHost);
    printf("Results: data[0]=%d, data[1]=%d, data[10]=%d\n",
           h_data[0], h_data[1], h_data[10]);
    
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc cg_grid_sync.cu -o cg_grid_sync && ./cg_grid_sync

## CG Reduction Operators

| Operator | Returns |
|----------|--------|
| `cg::plus<T>()` | Sum |
| `cg::less<T>()` | Minimum |
| `cg::greater<T>()` | Maximum |
| `cg::bit_and<T>()` | Bitwise AND |
| `cg::bit_or<T>()` | Bitwise OR |
| `cg::bit_xor<T>()` | Bitwise XOR |

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to practice advanced cooperative groups concepts:

**Exercise 1: Multi-Level Reduction**
- Implement a hierarchical reduction using both warp-level and block-level cooperative groups
- Use `cg::reduce()` for warp-level and shared memory for block-level

**Exercise 2: Warp-Aggregated Histogram**
- Implement a histogram using warp-aggregated atomics
- Compare performance with naive atomic operations

**Exercise 3: Grid-Wide Barrier**
- Implement a multi-pass algorithm using `grid.sync()`
- Example: multi-pass smoothing filter

In [ ]:
%%writefile advanced_cg_exercises.cu
#include <cuda_runtime.h>
#include <cooperative_groups.h>
#include <cooperative_groups/reduce.h>
#include <stdio.h>
#include <stdlib.h>

namespace cg = cooperative_groups;

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error at %s:%d: %s\n", __FILE__, __LINE__, \
                   cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

// =============================================================================
// Exercise 1: Multi-Level Hierarchical Reduction
// TODO: Complete the kernel to perform reduction at warp and block levels
// =============================================================================
__global__ void hierarchicalReduction(float* input, float* output, int n) {
    cg::thread_block block = cg::this_thread_block();
    cg::thread_block_tile<32> warp = cg::tiled_partition<32>(block);
    
    extern __shared__ float sdata[];
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load data
    float val = (gid < n) ? input[gid] : 0.0f;
    
    // TODO: Exercise 1a - Use cg::reduce() for warp-level reduction
    // Hint: float warp_sum = cg::reduce(warp, val, cg::plus<float>());
    float warp_sum = 0.0f;  // Replace with cg::reduce
    
    // TODO: Exercise 1b - Store warp results to shared memory (lane 0 only)
    // Hint: if (warp.thread_rank() == 0) sdata[warp_id] = warp_sum;
    
    block.sync();
    
    // TODO: Exercise 1c - Final reduction across warps (first warp only)
    // Reduce the warp sums stored in shared memory
    
    if (tid == 0) {
        output[blockIdx.x] = sdata[0];  // Store block result
    }
}

// =============================================================================
// Exercise 2: Warp-Aggregated Histogram
// TODO: Implement efficient histogram with leader election
// =============================================================================
#define NUM_BINS 256

__global__ void warpAggregatedHistogram(unsigned char* data, int* histogram, int n) {
    cg::thread_block block = cg::this_thread_block();
    cg::thread_block_tile<32> warp = cg::tiled_partition<32>(block);
    
    __shared__ int local_hist[NUM_BINS];
    
    // Initialize shared histogram
    for (int i = threadIdx.x; i < NUM_BINS; i += blockDim.x) {
        local_hist[i] = 0;
    }
    block.sync();
    
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (gid < n) {
        unsigned char bin = data[gid];
        
        // TODO: Exercise 2a - Find threads with matching bins using ballot
        // Hint: unsigned int mask = __ballot_sync(0xffffffff, true);
        //       unsigned int peers = ... find threads with same bin value
        
        // TODO: Exercise 2b - Count matching threads
        // Hint: int count = __popc(peers);
        
        // TODO: Exercise 2c - Elect leader (lowest lane) to do atomic
        // Hint: int leader = __ffs(peers) - 1;
        //       if (warp.thread_rank() == leader) atomicAdd(...)
        
        // Naive version (replace with aggregated version above):
        atomicAdd(&local_hist[bin], 1);
    }
    
    block.sync();
    
    // Merge to global histogram
    for (int i = threadIdx.x; i < NUM_BINS; i += blockDim.x) {
        if (local_hist[i] > 0) {
            atomicAdd(&histogram[i], local_hist[i]);
        }
    }
}

// =============================================================================
// Exercise 3: Grid-Wide Multi-Pass Smoothing
// TODO: Use grid.sync() for iterative algorithm
// =============================================================================
__global__ void multiPassSmooth(float* data, float* temp, int n, int passes) {
    cg::grid_group grid = cg::this_grid();
    
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    for (int pass = 0; pass < passes; pass++) {
        // TODO: Exercise 3a - Apply smoothing (average of neighbors)
        if (tid > 0 && tid < n - 1) {
            // Read from data, write to temp
            temp[tid] = (data[tid-1] + data[tid] + data[tid+1]) / 3.0f;
        } else if (tid < n) {
            temp[tid] = data[tid];  // Boundary
        }
        
        // TODO: Exercise 3b - Synchronize entire grid
        // Hint: grid.sync();
        
        // TODO: Exercise 3c - Swap pointers (copy temp back to data)
        if (tid < n) {
            data[tid] = temp[tid];
        }
        
        // TODO: Exercise 3d - Another grid sync before next pass
        grid.sync();
    }
}

// Test Exercise 1: Hierarchical Reduction
void testHierarchicalReduction() {
    printf("=== Exercise 1: Hierarchical Reduction ===\n");
    
    int n = 1024 * 1024;
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;
    
    float *h_input = (float*)malloc(n * sizeof(float));
    float *h_output = (float*)malloc(numBlocks * sizeof(float));
    
    float expected = 0.0f;
    for (int i = 0; i < n; i++) {
        h_input[i] = 1.0f;
        expected += h_input[i];
    }
    
    float *d_input, *d_output;
    CHECK_CUDA(cudaMalloc(&d_input, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_output, numBlocks * sizeof(float)));
    CHECK_CUDA(cudaMemcpy(d_input, h_input, n * sizeof(float), cudaMemcpyHostToDevice));
    
    int warpsPerBlock = blockSize / 32;
    int sharedSize = warpsPerBlock * sizeof(float);
    
    hierarchicalReduction<<<numBlocks, blockSize, sharedSize>>>(d_input, d_output, n);
    CHECK_CUDA(cudaGetLastError());
    
    CHECK_CUDA(cudaMemcpy(h_output, d_output, numBlocks * sizeof(float), cudaMemcpyDeviceToHost));
    
    float total = 0.0f;
    for (int i = 0; i < numBlocks; i++) {
        total += h_output[i];
    }
    
    printf("Expected: %.0f, Got: %.0f\n", expected, total);
    printf("Status: %s\n\n", (fabsf(total - expected) < 1e-3) ? "PASS" : "NEEDS WORK");
    
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);
}

// Test Exercise 2: Warp-Aggregated Histogram
void testWarpHistogram() {
    printf("=== Exercise 2: Warp-Aggregated Histogram ===\n");
    
    int n = 1024 * 1024;
    
    unsigned char *h_data = (unsigned char*)malloc(n);
    int *h_hist = (int*)calloc(NUM_BINS, sizeof(int));
    int expected[NUM_BINS] = {0};
    
    for (int i = 0; i < n; i++) {
        h_data[i] = rand() % NUM_BINS;
        expected[h_data[i]]++;
    }
    
    unsigned char *d_data;
    int *d_hist;
    CHECK_CUDA(cudaMalloc(&d_data, n));
    CHECK_CUDA(cudaMalloc(&d_hist, NUM_BINS * sizeof(int)));
    CHECK_CUDA(cudaMemcpy(d_data, h_data, n, cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemset(d_hist, 0, NUM_BINS * sizeof(int)));
    
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;
    
    warpAggregatedHistogram<<<numBlocks, blockSize>>>(d_data, d_hist, n);
    CHECK_CUDA(cudaGetLastError());
    
    CHECK_CUDA(cudaMemcpy(h_hist, d_hist, NUM_BINS * sizeof(int), cudaMemcpyDeviceToHost));
    
    bool correct = true;
    for (int i = 0; i < NUM_BINS; i++) {
        if (h_hist[i] != expected[i]) {
            correct = false;
            break;
        }
    }
    
    printf("Histogram check: %s\n", correct ? "PASS" : "NEEDS WORK");
    printf("Sample bins - [0]: expected=%d, got=%d\n\n", expected[0], h_hist[0]);
    
    cudaFree(d_data);
    cudaFree(d_hist);
    free(h_data);
    free(h_hist);
}

// Test Exercise 3: Grid-Wide Smoothing (requires cooperative launch)
void testGridSmooth() {
    printf("=== Exercise 3: Grid-Wide Smoothing ===\n");
    
    int deviceId;
    cudaGetDevice(&deviceId);
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, deviceId);
    
    if (!prop.cooperativeLaunch) {
        printf("Device does not support cooperative launch\n");
        printf("Status: SKIPPED\n\n");
        return;
    }
    
    int n = 1024;
    int passes = 3;
    
    float *h_data = (float*)malloc(n * sizeof(float));
    for (int i = 0; i < n; i++) {
        h_data[i] = (i == n/2) ? 100.0f : 0.0f;  // Spike in middle
    }
    
    float *d_data, *d_temp;
    CHECK_CUDA(cudaMalloc(&d_data, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_temp, n * sizeof(float)));
    CHECK_CUDA(cudaMemcpy(d_data, h_data, n * sizeof(float), cudaMemcpyHostToDevice));
    
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;
    
    void* args[] = {&d_data, &d_temp, &n, &passes};
    
    cudaError_t err = cudaLaunchCooperativeKernel(
        (void*)multiPassSmooth,
        dim3(numBlocks), dim3(blockSize),
        args, 0, 0
    );
    
    if (err == cudaSuccess) {
        CHECK_CUDA(cudaMemcpy(h_data, d_data, n * sizeof(float), cudaMemcpyDeviceToHost));
        
        printf("After %d smoothing passes:\n", passes);
        printf("Center region: [%.2f, %.2f, %.2f, %.2f, %.2f]\n",
               h_data[n/2-2], h_data[n/2-1], h_data[n/2], h_data[n/2+1], h_data[n/2+2]);
        printf("Status: Check if spike has spread\n\n");
    } else {
        printf("Cooperative launch failed: %s\n", cudaGetErrorString(err));
        printf("Status: NEEDS WORK\n\n");
    }
    
    cudaFree(d_data);
    cudaFree(d_temp);
    free(h_data);
}

int main() {
    printf("\n╔══════════════════════════════════════════════════════════════╗\n");
    printf("║     Advanced Cooperative Groups Exercises                    ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    testHierarchicalReduction();
    testWarpHistogram();
    testGridSmooth();
    
    printf("═══════════════════════════════════════════════════════════════\n");
    printf("Complete the TODOs in each exercise and re-run to verify!\n");
    printf("═══════════════════════════════════════════════════════════════\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -rdc=true -lcudadevrt -o advanced_cg_exercises advanced_cg_exercises.cu && ./advanced_cg_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises use Numba for GPU programming. While Numba doesn't support all cooperative groups features, you can practice the concepts:

```python
from numba import cuda
import numpy as np

# Exercise: Implement warp-level primitives simulation
# Numba supports some warp shuffle operations

@cuda.jit
def warp_reduce_sum(data, result):
    """
    Use cuda.shfl_down_sync to implement warp reduction
    Numba equivalent of cg::reduce for a warp
    """
    tid = cuda.threadIdx.x
    lane = tid % 32
    
    val = data[cuda.blockIdx.x * cuda.blockDim.x + tid]
    
    # TODO: Implement warp shuffle reduction
    # for offset in [16, 8, 4, 2, 1]:
    #     val += cuda.shfl_down_sync(0xffffffff, val, offset)
    
    if lane == 0:
        cuda.atomic.add(result, 0, val)
```

**Note**: For full cooperative groups functionality including `grid.sync()`, use CUDA C++.

## Key Takeaways

1. **`cg::reduce()`** - Hardware-accelerated on CC 8.0+
2. **`cg::inclusive_scan()` / `cg::exclusive_scan()`** - Parallel prefix operations
3. **Warp-aggregated atomics** - Reduce contention dramatically
4. **Grid sync** - Requires `cudaLaunchCooperativeKernel`
5. **Flexible partitioning** - `tiled_partition<N>()` for any power-of-2